# Source reconstruction 

The aim of this lecture is to teach you how to compute and apply
a linear inverse method. We especially look into how to compute and apply a beamformer spatial filter. Later, there is also information on how to compute a minimum norm estimation model.

`
Authors: Britta Westner, Alexandre Gramfort, Denis Engemann 
`

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import numpy as np
import mne

mne.set_log_level('warning')

# Change the following path to where the ds000117 and the extra folder are on your disk
data_path = os.path.expanduser("~/Documents/teaching/2023_berlin_brains/data")
epochs_fname = os.path.join(data_path, 'sub-02/sub-02-epo.fif')


## Read epochs and compute ERF

In [ ]:
epochs = mne.read_epochs(epochs_fname)
epochs.info

In [ ]:
# since we computed our forward model for MEG only, we drop the EEG channels
epochs.pick_types(meg=True, eeg=False)

Let's compute the evoked responses for two conditions: _faces_  and  _scrambled_

In [ ]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

Compute the contrast between the two conditions and look at it

In [ ]:
evoked_contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])
evoked_contrast.crop(-0.05, 0.25)
evoked_contrast.plot();

## Prepare beamforming of data


### Compute data covariance matrix
For beamforming, we need a **data covariance matrix**.

Since we want to contrast conditions, we will compute a so-called **common spatial filter** - meaning we will use a covariance matrix that was computed on both conditions jointly. In our case, that is all the data (faces + scrambled).

In [ ]:
data_cov = mne.compute_covariance(epochs, tmin=0.1, tmax=0.25,
                                  method='empirical', rank='info')

Let's visualize our covariance matrices.

In [ ]:
mne.viz.plot_cov(data_cov, info=epochs.info)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>This data is severely rank-deficient. Do you know where to see this? </li>
      <li>Can you guess why that is?</li>     
      <li>Do you know what that means for our beamformer?</li>
    </ul>
</div>

Let's keep the rank information around in a dictionary:

In [ ]:
ranks = {'mag': 68, 'grad': 68}  

### Compute noise covariance matrix

Since we have two sensor types that we want to combine (gradiometers and magnetometers), we also need to compute a noise covariance matrix. That will be used for pre-whitening the data, data covariance matrix, and forward model. This is needed to take care of the different orders of magnitudes of the sensor types.

In [ ]:
noise_cov = mne.compute_covariance(epochs, 
                                   tmin=-.05, tmax=0.05,  # use as baseline
                                   method='empirical', 
                                   rank='info')

Lastly, we also need to read the forward model that we had saved!

In [ ]:
fwd_fname = os.path.join(data_path,
    'sub-02/sub-02-meg-fwd.fif')
fwd = mne.read_forward_solution(fwd_fname)

# Restrict forward solution to MEG channels only
fwd = mne.pick_types_forward(fwd, meg=True, eeg=False)

## Compute beamformer and apply to evoked data

Now we can compute the spatial filter (beamformer):

In [ ]:
from mne.beamformer import make_lcmv, apply_lcmv

In [ ]:
filters = make_lcmv(
    epochs.info, fwd,
    data_cov=data_cov, noise_cov=noise_cov,
    pick_ori='max-power', rank=ranks
)

We can apply the filter to one of our conditions to see the activation following picture presentation:

In [ ]:
stc_face = apply_lcmv(evoked=evoked_face, filters=filters)

We can plot the brain and time course using `stc_face.plot()`. You can explore the source reconstruction, e.g. by watching the activation as a movie. 

We crop the `stc` object in time.

In [ ]:
subjects_dir = os.path.join(data_path, 'freesurfer')
stc_face.crop(-0.05, 0.25).plot(subjects_dir=subjects_dir, subject='sub-02', hemi='both')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Discuss the reconstruction: how does the activity spread?</li>
      <li> Do you know what do the negative and positive activations mean? </li>
    </ul>
</div>

Let's apply the same filter to our other condition. We can then subtract the activity from each other.

In [ ]:
%matplotlib qt
stc_scrambled = apply_lcmv(evoked=evoked_scrambled, filters=filters)
stc_scrambled.crop(-0.05, 0.25)

In [ ]:
stc_diff = stc_face.copy()
stc_diff.data = np.abs(stc_face.data) - np.abs(stc_scrambled.data)

stc_diff.plot(subjects_dir=subjects_dir, subject='sub-02', hemi='both')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Do the negative and positive activations mean the same as before?  </li>
      <li>Can you save a screenshot of the activity at 150 ms?  </li>
    </ul>
</div>

## Bonus epsiode: Minimum norm estimation (MNE)

### Prepare the MNE solution

To compute our inverse operator for minimum norm estimation, we need a noise covariance matrix. We could just use the one we already computed, but let's try something new: we compute the covariance matrix with different methods and let the algorithm choose the best one!

For more information, check out:

`Engemann DA & Gramfort A (2015): Automated model selection in covariance estimation and spatial whitening of MEG and EEG signals, NeuroImage. `

In [ ]:
noise_cov = mne.compute_covariance(epochs, tmax=0.05,
                                   method=['shrunk', 'empirical'],
                                   rank='info')
noise_cov['method']

We can visualize the whitening of the evoked data, using this noise covariance matrix:

In [ ]:
%matplotlib inline
evoked_contrast.plot_white(noise_cov);

For the MNE source solution, we want to use a fixed forward operator:

In [ ]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True)

### Compute MNE inverse operator and apply to evoked data

In [ ]:
from mne.minimum_norm import (make_inverse_operator, apply_inverse)

Minimum norm inverse models are independant
from the data (as they use just the noise covariance but not a data covariance matrix) and can therefore be
precomputed and applied to the data at a later stage.

We do not need to take special care of our conditions here.

In [ ]:
info = evoked_contrast.info
inverse_operator = make_inverse_operator(info, fwd_fixed, noise_cov,
                                         loose=0.2, depth=0.8)

Now let's apply this inverse operator to our evoked contrast:

In [ ]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2  # regularization
stc = apply_inverse(evoked_contrast, inverse_operator, lambda2,
                    method=method, pick_ori=None)
print(stc)

Let us plot the results just as before with the beamformer:

In [ ]:
subjects_dir = os.path.join(data_path, 'freesurfer')
stc.plot(hemi='both', subjects_dir=subjects_dir, subject='sub-02')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Compare the MNE reconstruction to the beamformer. What differences can you see? Can you explain them? </li>     
      <li>Run sLORETA on the same data and compare source localizations. </li>
    </ul>
</div>
